# Fetch player headshot from Wikidata (P18)
Steps: search for the player -> get QID -> fetch P18 image filename -> build Commons URL.
Uses Wikimedia APIs (no scraping).


In [4]:
import requests, urllib.parse

session = requests.Session()
session.headers.update({
    "User-Agent": "PSLStats/0.1 (contact: you@example.com)",
    "Accept": "application/json",
})

player_name = "Babar Azam"
search_url = "https://www.wikidata.org/w/api.php"
entity_url = search_url

def search_qid(name: str):
    params = {
        "action": "wbsearchentities",
        "search": name,
        "language": "en",
        "format": "json",
        "limit": 1,
    }
    r = session.get(search_url, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    return data.get("search", [{}])[0].get("id")

def fetch_p18(qid: str):
    params = {
        "action": "wbgetentities",
        "ids": qid,
        "props": "claims",
        "format": "json",
    }
    r = session.get(entity_url, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    try:
        return data["entities"][qid]["claims"]["P18"][0]["mainsnak"]["datavalue"]["value"]
    except Exception:
        return None

qid = search_qid(player_name)
filename = fetch_p18(qid) if qid else None
print("QID:", qid, "P18:", filename)
if filename:
    encoded = urllib.parse.quote(filename)
    print("Image:", f"https://commons.wikimedia.org/wiki/Special:FilePath/{encoded}?width=250")
else:
    print("No image found; use placeholder")


QID: Q4837515 P18: Babar azam 2023.jpg
Image: https://commons.wikimedia.org/wiki/Special:FilePath/Babar%20azam%202023.jpg?width=250


In [7]:
import requests, urllib.parse

session = requests.Session()
session.headers.update({
    "User-Agent": "PSLStats/0.1 (contact: you@example.com)",
    "Accept": "application/json",
})

player_name = "Babar Azam"
search_url = "https://www.wikidata.org/w/api.php"
entity_url = search_url

def search_qid(name: str):
    params = {
        "action": "wbsearchentities",
        "search": name,
        "language": "en",
        "format": "json",
        "limit": 1,
    }
    r = session.get(search_url, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    return data.get("search", [{}])[0].get("id")

def fetch_p18(qid: str):
    params = {
        "action": "wbgetentities",
        "ids": qid,
        "props": "claims",
        "format": "json",
    }
    r = session.get(entity_url, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    try:
        return data["entities"][qid]["claims"]["P18"][0]["mainsnak"]["datavalue"]["value"]
    except Exception:
        return None

qid = search_qid(player_name)
filename = fetch_p18(qid) if qid else None
print("QID:", qid, "P18:", filename)
if filename:
    encoded = urllib.parse.quote(filename)
    print("Image:", f"https://commons.wikimedia.org/wiki/Special:FilePath/{encoded}?width=250")
else:
    print("No image found; use placeholder")


QID: Q4837515 P18: Babar azam 2023.jpg
Image: https://commons.wikimedia.org/wiki/Special:FilePath/Babar%20azam%202023.jpg?width=250


In [ ]:
re

QID: Q4837515 P18: Babar azam 2023.jpg


ConnectionError: HTTPSConnectionPool(host='commons.wikimedia.org', port=443): Max retries exceeded with url: /w/api.php?action=query&titles=File%3ABabar+azam+2023.jpg&prop=imageinfo&iiprop=url&iiurlwidth=250&format=json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000257295032D0>: Failed to resolve 'commons.wikimedia.org' ([Errno 11001] getaddrinfo failed)"))

In [1]:
# Install required packages first (run this in a separate cell if needed):
# !pip install requests beautifulsoup4 pillow

import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
from PIL import Image
from io import BytesIO
import os

def download_player_image(player_name, num_images=1, save_folder="downloads"):
    """
    Download cricket player images from Google Images
    
    Args:
        player_name: Name of the cricket player
        num_images: Number of images to download (default: 1)
        save_folder: Folder to save images (default: "downloads")
    """
    
    # Create download folder if it doesn't exist
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    
    # Prepare search query
    search_query = quote(f"{player_name} cricket player")
    url = f"https://www.google.com/search?q={search_query}&tbm=isch"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        # Fetch Google Images page
        print(f"Searching for {player_name}...")
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find image URLs
        img_tags = soup.find_all('img')
        downloaded_count = 0
        
        for idx, img in enumerate(img_tags[1:]):  # Skip first image (Google logo)
            if downloaded_count >= num_images:
                break
                
            img_url = img.get('src') or img.get('data-src')
            
            if img_url and img_url.startswith('http'):
                try:
                    # Download image
                    print(f"Downloading image {downloaded_count + 1}...")
                    img_response = requests.get(img_url, headers=headers, timeout=10)
                    img_response.raise_for_status()
                    
                    # Open and save image
                    image = Image.open(BytesIO(img_response.content))
                    filename = f"{player_name.replace(' ', '_')}_{downloaded_count + 1}.jpg"
                    filepath = os.path.join(save_folder, filename)
                    
                    # Convert to RGB if necessary (for PNG with transparency)
                    if image.mode in ('RGBA', 'LA', 'P'):
                        image = image.convert('RGB')
                    
                    image.save(filepath, 'JPEG')
                    print(f"✓ Saved: {filepath}")
                    downloaded_count += 1
                    
                except Exception as e:
                    print(f"✗ Failed to download image {idx}: {str(e)}")
                    continue
        
        if downloaded_count > 0:
            print(f"\n✓ Successfully downloaded {downloaded_count} image(s) to '{save_folder}' folder")
        else:
            print("✗ No images were downloaded. Try a different player name.")
            
    except Exception as e:
        print(f"✗ Error: {str(e)}")

# Usage Example:
# Download 1 image of Babar Azam
download_player_image("Kamran Akmal", num_images=1)

# Download 3 images of Virat Kohli
# download_player_image("Virat Kohli", num_images=3)

# Download to custom folder
# download_player_image("Babar Azam", num_images=1, save_folder="cricket_players")

Searching for Kamran Akmal...
✓ Saved: downloads\Kamran_Akmal_1.jpg

✓ Successfully downloaded 1 image(s) to 'downloads' folder


In [23]:
# Prereqs (run once separately):
# !pip install requests beautifulsoup4 pillow

import os, time, csv, random
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
from PIL import Image
from io import BytesIO
from pathlib import Path

DATA_PATH = Path(r"E:\psl\Data\PSL_Complete_Dataset_2016_2025.csv")
SAVE_FOLDER = Path("downloads_psl_players")
SAVE_FOLDER.mkdir(exist_ok=True)

# Log files to track progress
SUCCESS_LOG = SAVE_FOLDER / "success.txt"
FAILED_LOG = SAVE_FOLDER / "failed.txt"

def load_processed():
    """Load already processed players"""
    processed = set()
    for log in [SUCCESS_LOG, FAILED_LOG]:
        if log.exists():
            with log.open("r", encoding="utf-8") as f:
                processed.update(line.strip() for line in f if line.strip())
    return processed

def log_result(name: str, status: str):
    """Log result: 'success' or 'failed'"""
    log_map = {"success": SUCCESS_LOG, "failed": FAILED_LOG}
    with log_map[status].open("a", encoding="utf-8") as f:
        f.write(f"{name}\n")

def download_player_image(player_name, save_folder):
    """
    Download one cricket player image from Google Images
    Returns True on success, False on failure
    """
    
    # Check if file already exists
    filename = f"{player_name.replace(' ', '_')}.jpg"
    filepath = save_folder / filename
    
    if filepath.exists():
        print(f"✓ (already exists)")
        return True
    
    # Prepare search query
    search_query = quote(f"{player_name} cricket player")
    url = f"https://www.google.com/search?q={search_query}&tbm=isch"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        # Fetch Google Images page
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find image URLs
        img_tags = soup.find_all('img')
        
        for idx, img in enumerate(img_tags[1:10]):  # Skip first image (Google logo), try first 10
            img_url = img.get('src') or img.get('data-src')
            
            if img_url and img_url.startswith('http'):
                try:
                    # Download image
                    img_response = requests.get(img_url, headers=headers, timeout=15)
                    img_response.raise_for_status()
                    
                    # Open and save image
                    image = Image.open(BytesIO(img_response.content))
                    
                    # Skip tiny images
                    if image.width < 80 or image.height < 80:
                        continue
                    
                    # Convert to RGB if necessary
                    if image.mode in ('RGBA', 'LA', 'P'):
                        image = image.convert('RGB')
                    
                    image.save(filepath, 'JPEG', quality=90)
                    print(f"✓")
                    return True
                    
                except Exception:
                    continue
        
        print(f"✗")
        return False
            
    except Exception as e:
        print(f"✗ (error)")
        return False

# Load unique players from CSV
players = []
if DATA_PATH.exists():
    with DATA_PATH.open(newline="", encoding="utf-8", errors="ignore") as f:
        reader = csv.DictReader(f)
        seen = set()
        for row in reader:
            n = (row.get("batter") or "").strip()
            if n and n not in seen:
                seen.add(n)
                players.append(n)
else:
    print(f"❌ CSV not found: {DATA_PATH}")
    exit()

# Load processed players
processed = load_processed()
remaining = [p for p in players if p not in processed]

print(f"\n{'='*70}")
print(f"🏏 PSL PLAYER IMAGE DOWNLOADER")
print(f"{'='*70}")
print(f"📊 Total unique batters: {len(players)}")
print(f"✓ Already processed: {len(processed)}")
print(f"⏳ Remaining: {len(remaining)}")
print(f"📁 Save folder: {SAVE_FOLDER}")
print(f"⏱️  Delay: 5 seconds between downloads")
print(f"{'='*70}\n")

if not remaining:
    print("✅ All players already processed!")
else:
    print("🚀 Starting downloads... (Ctrl+C to pause anytime)\n")
    
    success = 0
    failed = 0
    
    try:
        for idx, name in enumerate(remaining, 1):
            print(f"[{idx}/{len(remaining)}] {name:<30} ... ", end="", flush=True)
            
            # Download image
            ok = download_player_image(name, SAVE_FOLDER)
            
            # Log result
            if ok:
                log_result(name, "success")
                success += 1
            else:
                log_result(name, "failed")
                failed += 1
            
            # Wait 5 seconds before next download (except for last one)
            if idx < len(remaining):
                time.sleep(5)
            
            # Progress summary every 20 players
            if idx % 20 == 0:
                print(f"\n{'─'*70}")
                print(f"Progress: {idx}/{len(remaining)} | ✓ {success} | ✗ {failed}")
                print(f"{'─'*70}\n")
                
    except KeyboardInterrupt:
        print("\n\n⚠ Stopped by user (Ctrl+C)")
    
    finally:
        print(f"\n{'='*70}")
        print(f"📊 FINAL SUMMARY")
        print(f"{'='*70}")
        print(f"✓ Successfully downloaded: {success}")
        print(f"✗ Failed: {failed}")
        print(f"📁 Images location: {SAVE_FOLDER.absolute()}")
        print(f"📝 Check logs: success.txt, failed.txt")
        print(f"\n💡 Re-run this cell anytime to resume!")
        print(f"{'='*70}")


🏏 PSL PLAYER IMAGE DOWNLOADER
📊 Total unique batters: 380
✓ Already processed: 2
⏳ Remaining: 378
📁 Save folder: downloads_psl_players
⏱️  Delay: 5 seconds between downloads

🚀 Starting downloads... (Ctrl+C to pause anytime)

[1/378] DJ Malan                       ... ✓
[2/378] EJG Morgan                     ... ✓
[3/378] Haris Sohail                   ... ✓
[4/378] DJG Sammy                      ... ✓
[5/378] Shahid Afridi                  ... ✓
[6/378] CJ Jordan                      ... ✓
[7/378] Iftikhar Ahmed                 ... ✓
[8/378] Hasan Ali                      ... ✓
[9/378] DR Smith                       ... ✓
[10/378] Sharjeel Khan                  ... ✓
[11/378] BJ Haddin                      ... ✓
[12/378] SR Watson                      ... ✓
[13/378] SW Billings                    ... ✓
[14/378] Asad Shafiq                    ... ✓
[15/378] Ahmed Shehzad                  ... ✓
[16/378] KP Pietersen                   ... ✓
[17/378] RR Rossouw                     ... ✓
